In [78]:
import pandas as pd
import numpy as np

In [79]:
data=pd.read_csv("./data/gemstone.csv")

In [80]:
data.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,124589,2.20,Very Good,H,SI2,61.7,59.0,8.34,8.41,5.18,18001
1,193368,0.52,Very Good,D,VS2,63.2,58.0,5.08,5.11,3.22,1692
2,88147,0.47,Premium,F,SI2,59.7,61.0,5.04,5.01,3.01,984
3,153848,1.26,Ideal,G,VS1,61.1,57.0,7.03,7.01,4.27,8703
4,80008,0.70,Very Good,G,VS2,61.9,61.0,5.69,5.74,3.53,2648


In [81]:
data.drop(labels=["id"],axis=1,inplace=True)

In [82]:
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,2.20,Very Good,H,SI2,61.7,59.0,8.34,8.41,5.18,18001
1,0.52,Very Good,D,VS2,63.2,58.0,5.08,5.11,3.22,1692
2,0.47,Premium,F,SI2,59.7,61.0,5.04,5.01,3.01,984
3,1.26,Ideal,G,VS1,61.1,57.0,7.03,7.01,4.27,8703
4,0.70,Very Good,G,VS2,61.9,61.0,5.69,5.74,3.53,2648


create model we need independent and dependent variables

In [83]:
x=data.drop(labels=["price"],axis=1)


In [84]:
x

,carat,cut,color,clarity,depth,table,x,y,z
0,2.20,Very Good,H,SI2,61.7,59.0,8.34,8.41,5.18
1,0.52,Very Good,D,VS2,63.2,58.0,5.08,5.11,3.22
2,0.47,Premium,F,SI2,59.7,61.0,5.04,5.01,3.01
3,1.26,Ideal,G,VS1,61.1,57.0,7.03,7.01,4.27
4,0.70,Very Good,G,VS2,61.9,61.0,5.69,5.74,3.53
...,...,...,...,...,...,...,...,...,...
145174,0.79,Ideal,I,SI1,62.2,56.0,5.98,5.95,3.69
145175,2.17,Ideal,I,SI2,61.9,55.0,8.29,8.33,5.16
145176,0.75,Very Good,F,VS2,62.7,59.0,5.74,5.79,3.61
145177,1.00,Very Good,F,SI2,62.5,60.0,6.34,6.31,3.96


In [85]:
y=data["price"]

In [86]:
y

0         18001
1          1692
2           984
3          8703
4          2648
          ...  
145174     2789
145175    13393
145176     2937
145177     4541
145178     3256
Name: price, Length: 145179, dtype: int64

In [87]:
categorical_cols=x.select_dtypes(include="object").columns
categorical_cols

Index(['cut', 'color', 'clarity'], dtype='object')

In [88]:
numerical_cols=x.select_dtypes(exclude="object").columns

In [89]:
numerical_cols

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [90]:
numerical_cols=x.select_dtypes(exclude="object")

In [91]:
numerical_cols

,carat,depth,table,x,y,z
0,2.20,61.7,59.0,8.34,8.41,5.18
1,0.52,63.2,58.0,5.08,5.11,3.22
2,0.47,59.7,61.0,5.04,5.01,3.01
3,1.26,61.1,57.0,7.03,7.01,4.27
4,0.70,61.9,61.0,5.69,5.74,3.53
...,...,...,...,...,...,...
145174,0.79,62.2,56.0,5.98,5.95,3.69
145175,2.17,61.9,55.0,8.29,8.33,5.16
145176,0.75,62.7,59.0,5.74,5.79,3.61
145177,1.00,62.5,60.0,6.34,6.31,3.96


In [127]:
#define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [128]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [129]:
num_pipeline=Pipeline(
    
    
    steps=[
        
        ('imputer',SimpleImputer()),
        ('scaler', StandardScaler())
    ]
    
    
)


In [130]:
cat_pipeline=Pipeline(
    
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories]))
    ]
    
)

In [137]:
preprocessor=ColumnTransformer(
    [
        
        ('num_pipeline',num_pipeline,['carat', 'depth', 'table', 'x', 'y', 'z']),
        ('cat_pipeline',cat_pipeline,['cut', 'color', 'clarity'])
    ]
)

Train test split

In [138]:
# Train test split

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=30)

In [139]:
preprocessor.fit_transform(x_train)

array([[ 0.23521783, -3.44063807,  3.01303699, ...,  1.        ,
         1.        ,  2.        ],
       [ 0.92698491,  1.09097884,  0.9242457 , ...,  3.        ,
         4.        ,  3.        ],
       [ 0.47301276, -0.38873281,  0.40204788, ...,  3.        ,
         0.        ,  2.        ],
       ...,
       [-1.12669862,  0.07367708, -0.64234777, ...,  4.        ,
         1.        ,  3.        ],
       [-0.58625559, -0.29625083, -1.16454559, ...,  4.        ,
         1.        ,  3.        ],
       [-0.60787331, -0.57369676, -1.16454559, ...,  4.        ,
         4.        ,  6.        ]])

In [140]:
preprocessor.transform(x_test) # only tranform

array([[ 0.64595453, -0.8511427 ,  0.40204788, ...,  3.        ,
         3.        ,  3.        ],
       [-0.84566824,  0.35112301, -0.64234777, ...,  4.        ,
         1.        ,  3.        ],
       [-0.91052141, -1.86844445,  0.40204788, ...,  3.        ,
         3.        ,  5.        ],
       ...,
       [ 0.45139504,  1.55338873, -0.64234777, ...,  2.        ,
         2.        ,  1.        ],
       [ 0.12712922, -0.0188049 , -0.64234777, ...,  4.        ,
         0.        ,  1.        ],
       [ 0.88374947,  0.16615906, -1.16454559, ...,  4.        ,
         4.        ,  4.        ]])

In [141]:
preprocessor.get_feature_names_out()

array(['num_pipeline__carat', 'num_pipeline__depth',
       'num_pipeline__table', 'num_pipeline__x', 'num_pipeline__y',
       'num_pipeline__z', 'cat_pipeline__cut', 'cat_pipeline__color',
       'cat_pipeline__clarity'], dtype=object)

In [142]:
x_train=pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())
x_test=pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())

In [143]:
x_train

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.235218,-3.440638,3.013037,0.535028,0.515789,0.254956,1.0,1.0,2.0
1,0.926985,1.090979,0.924246,1.004084,0.969655,1.086225,3.0,4.0,3.0
2,0.473013,-0.388733,0.402048,0.625231,0.588407,0.575796,3.0,0.0,2.0
3,-0.824051,0.443605,-1.164546,-0.836061,-0.863964,-0.809652,4.0,4.0,2.0
4,0.235218,1.368425,0.402048,0.318540,0.370552,0.473711,2.0,5.0,2.0
...,...,...,...,...,...,...,...,...,...
101620,-0.543020,-0.111287,-0.642348,-0.421126,-0.401020,-0.415893,4.0,2.0,5.0
101621,-1.018610,-0.111287,-0.642348,-1.151772,-1.181670,-1.174244,4.0,0.0,3.0
101622,-1.126699,0.073677,-0.642348,-1.413361,-1.399525,-1.378416,4.0,1.0,3.0
101623,-0.586256,-0.296251,-1.164546,-0.511329,-0.491793,-0.517979,4.0,1.0,3.0


In [144]:
x_test

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.645955,-0.851143,0.402048,0.895840,0.833495,0.779968,3.0,3.0,3.0
1,-0.845668,0.351123,-0.642348,-0.863122,-0.891195,-0.867987,4.0,1.0,3.0
2,-0.910521,-1.868444,0.402048,-0.872142,-0.918427,-1.042991,3.0,3.0,5.0
3,2.634785,1.090979,0.924246,2.158685,2.122474,2.267502,3.0,6.0,4.0
4,0.473013,2.200763,0.924246,0.562089,0.533944,0.779968,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...
43549,1.532281,0.443605,0.924246,1.428039,1.378134,1.450817,3.0,4.0,4.0
43550,-1.061845,-0.388733,1.446444,-1.241975,-1.290597,-1.290914,3.0,1.0,2.0
43551,0.451395,1.553389,-0.642348,0.553068,0.506712,0.677882,2.0,2.0,1.0
43552,0.127129,-0.018805,-0.642348,0.291479,0.316088,0.313290,4.0,0.0,1.0


## model building

linear regression

ridge regression

lasso regression

elastic net

## Model Training

In [145]:


from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [146]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

## Train multiple models

In [147]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

In [148]:
trained_model_list=[]
model_list=[]
r2_list=[]

In [149]:
list(models)

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']

In [150]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    print(model)

LinearRegression()
Lasso()
Ridge()
ElasticNet()


In [151]:
models.keys()

dict_keys(['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet'])

In [152]:
models.values()

dict_values([LinearRegression(), Lasso(), Ridge(), ElasticNet()])

## fit the model

In [154]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    #Make Predictions
    y_pred=model.predict(x_test)

    #this is a validation(test) score
    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 1027.6732577143728
MAE: 679.5866489945122
R2 score 93.54130202062986


Lasso
Model Training Performance
RMSE: 1024.3111696812239
MAE: 680.4284459538338
R2 score 93.58349284461762


Ridge
Model Training Performance
RMSE: 1027.70751292115
MAE: 679.6288562720443
R2 score 93.54087144073402


Elasticnet
Model Training Performance
RMSE: 1532.6801530862458
MAE: 1061.6825223811109
R2 score 85.63393148969499




In [155]:
r2_list

[0.9354130202062986,
 0.9358349284461762,
 0.9354087144073402,
 0.8563393148969499]